# Datos de nutrición del restaurante Subway

# Ejercicio: Técnicas de Análisis

En esta práctica vamos a realizar un estudio de los indicadores nutricionales de los diferentes productos vendidos en Subway, la conocida empresa de comida rápida especializada en sandwiches y bocadillos.

Datos fuente:

https://www.kaggle.com/davinm/subway-restaurant-nutrition-data

También se pueden ver directamente en su web oficial: https://www.subway.com/en-US/MenuNutrition/Nutrition/NutritionGrid

# Descripción de los datos

El conjunto de datos que vamos a utilizar contiene preparados según la receta estándar (pan de trigo de 9 granos con lechuga, tomates, cebollas, pimientos verdes y pepinos). La información nutricional de todos los bocadillos se basa en las recetas recomendadas por el chef. Dsiponemos de las siguientes caracerísticas para cada uno de los preparados:

`Product`: Nombre del preparado.

`Category`: Categoría a la que pertenece: Sandwich, Salad, Breakfast o 'Wrap', entre otros.

`Serving Size (g)`: Peso del preparado.

`Calories`: Calorías.

`Total Fat (g)`: Grasas total.

`Saturated Fat (g)`: Grasas saturadas.

`Trans Fat (g)`: Ácido graso.

`Cholesterol (mg)`: Colesterol.

`Sodium (mg)`: Sodio.

`Carbohydrates (g)`: Hidratos de carbono.

`Dietary Fiber (g)`: Fibra.

`Sugars (g)`: Azúcares.

`Protein (g)`: Proteínas.

`Vitamin A % DV`: Vitamina A.

`Vitamin C % DV`: Vitamina C.

`Calcium % DV`: Calcio.

`Iron % DV`. Hierro.



![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

# Importamos todas las librerías necesarias para realizar el estudio

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, RepeatedKFold, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_score
from sklearn.tree import DecisionTreeClassifier, plot_tree

import warnings
warnings.filterwarnings("ignore")

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

# Análisis exploratorio

Importamos el dataset, en formato csv, a un dataframe de la librería Pandas.

In [ ]:
subway = pd.read_csv('../input/subway-restaurant-nutrition-data/exported_data.csv', sep=",", dtype=str, encoding='iso-8859-1')
subway.head()

Renombramos la columna "Unnamed: 0", que contiene el nombre del preparado, por "Product".

In [ ]:
subway = subway.rename(columns={'Unnamed: 0':'Product'})

Vemos cuántos productos distintos se preparan en el restaurante, y el número de características total (entre las que se encuentran el nombre del preparado y la categoría), como valores discretos (categorías), y sus características nutricionales (valores numéricos). Como se puede observar, disponemos de información de 135 preparados con 15 valores nutricionales asociados (especificados anteriormente en la descripción de los datos), además del nombre y la categoría mencionados.

In [ ]:
subway.shape

Revisamos la distribución de preparados por categorías. 

In [ ]:
pd.DataFrame(subway['Category'].value_counts(dropna=False))

Vemos que lo que la categoría que más variedad de preparados tiene son los sandwiches, seguido de los rollos (wraps) y las ensaladas.

Vemos la distribución en una gráfica de barras.

In [ ]:
plt.figure(figsize=(16, 7))
sns.countplot(subway["Category"])
plt.show()

## Valores ausentes

Comprobamos si en el dataset existen valores no informados en alguna de las características. Como se puede observar, no encontramos ninguno, por lo que no necesitamos realizar ninguna acción al respecto.

In [ ]:
subway.isnull().sum()

Comprobamos el tipo de datos de cada característica:

In [ ]:
subway.dtypes

Como se puede observar, a pesar de que tenemos valores numéricos en todas las características correspondientes a la información nutricional de los preparados, al importar los datos desde csv las columnas (series) del dataframe se crearon como objetos. Los convertimos:

In [ ]:
subway['Serving Size (g)'] = subway['Serving Size (g)'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Calories'] = subway['Calories'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Total Fat (g)'] = subway['Total Fat (g)'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Saturated Fat (g)'] = subway['Saturated Fat (g)'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Trans Fat (g)'] = subway['Trans Fat (g)'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Cholesterol (mg)'] = subway['Cholesterol (mg)'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Sodium (mg)'] = subway['Sodium (mg)'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Carbohydrates (g)'] = subway['Carbohydrates (g)'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Dietary Fiber (g)'] = subway['Dietary Fiber (g)'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Sugars (g)'] = subway['Sugars (g)'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Protein (g)'] = subway['Protein (g)'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Vitamin A % DV'] = subway['Vitamin A % DV'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Vitamin C % DV'] = subway['Vitamin C % DV'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Calcium % DV'] = subway['Calcium % DV'].str.replace('.', '').str.replace(',', '.').astype(float)
subway['Iron % DV'] = subway['Iron % DV'].str.replace('.', '').str.replace(',', '.').astype(float)

Y comprobamos si el cambio ha surtido efecto:

In [ ]:
subway.dtypes

Para finalizar la parte de análisis exploratorio, mostramos los estadísticos más relevantes de los valores nutricionales:

In [ ]:
subway.describe()

Y vemos, por ejemplo, cómo se distribuyen las calorías en un diagrama de caja con un diagrama de enjambre superpuesto para cada una de las categorías.

In [ ]:
subway_cal = subway[['Product', 'Category', 'Calories']].pivot(index=['Product'], columns='Category', values='Calories').reset_index()

plt.figure(figsize=(20, 10))
sns.boxplot(data=subway_cal)
sns.stripplot(data=subway_cal, color='darkblue', alpha=0.7)
plt.show()

## Matriz de correlación

La matriz de correlación nos permite comprender la relación entre las diferentes características de nuestro conjunto de datos. Cada fila y columna representa una variable, y cada valor de esta matriz es el coeficiente de correlación entre las variables representadas por la fila y columna correspondientes.

El valor de cada casilla representa la correlación entre pares de variables:

* Un valor positivo grande (cercano a 1,0) indica una fuerte correlación positiva, es decir, si el valor de una de las variables aumenta, el valor de la otra variable aumenta también.

* Un valor negativo grande (cercano a -1,0) indica una fuerte correlación negativa, es decir, que el valor de una de las variables disminuye al aumentar el de la otra y viceversa.

* Un valor cercano a 0 (tanto positivo como negativo) indica la ausencia de cualquier correlación entre las dos variables, y por lo tanto esas variables son independientes entre sí.

Esta información es importante para el preprocesamiento en aprendizaje automático cuando se desea reducir la dimensionalidad de un dato de alta dimensión (no lo vemos aquí).


In [ ]:
mat = subway[['Serving Size (g)', 'Calories', 'Total Fat (g)', 'Saturated Fat (g)', 'Trans Fat (g)'
            , 'Cholesterol (mg)', 'Sodium (mg)', 'Carbohydrates (g)', 'Dietary Fiber (g)', 'Sugars (g)'
            , 'Protein (g)', 'Vitamin A % DV', 'Vitamin C % DV', 'Calcium % DV', 'Iron % DV']].corr().abs()

mask = np.triu(np.ones_like(mat, dtype=bool))
mat_masked = mat.mask(mask)  # Pone a NaN todo lo que aparezca como True en la máscara

fig, ax = plt.subplots(figsize=(10,7)) 
sns.heatmap(mat_masked, annot=True, ax=ax)
plt.show()

Por poner un ejemplo, se puede observar una alta correlación en las calorías respecto al tamaño del preparado o a los carbohidratos, de un 75% y un 86%, respectivamente. Parece claro que cuanto más grande es un preparado o más hidratos de carbono tiene, la cantidad de calorías será mayor. Otro ejemplo podría ser la alta correlación entre proteína y colesterol.

Este es el punto de partida, pero puede ocurrir que obtengamos valores que nos lleven a pensar que existe cierta relación pero que no sean consecuencia directa, sobre todo en conjuntos de datos pequeños, por lo que habría que estudiarlas en más profundidad.

Vemos ahora cómo correlacionan las calorías y el tamaño con los diferentes tipos de preparados.

En la creación de la matriz de correlación, las variables cualitativas no nos van a aportar información sobre su posible correlación con el resto de campos numéricos. Para poder manejarlo, hacemos las modificaciones necesarias para convertirlas en variables binarias, que sí nos ofrecen esa posibilidad.

Para ello, creamos la función get_categories para pasar las posibles categorías de los preparados a columnas binarias.

In [ ]:
def get_categories(data):

    cat_Sandwich = False
    cat_Salad = False
    cat_Breakfast = False
    cat_Extra = False
    cat_Wrap = False
    cat_Bread = False
    cat_Cheese = False
    cat_Extras = False
    cat_Sauces = False
    cat_Veggies = False
    cat_Protein = False
    cat_Seasonings = False

    if data['Category'] == 'Sandwich':
        cat_Sandwich = True
    elif data['Category'] == 'Salad':
        cat_Salad = True
    elif data['Category'] == 'Breakfast':
        cat_Breakfast = True
    elif data['Category'] == 'Extra':
        cat_Extra = True
    elif data['Category'] == 'Wrap':
        cat_Wrap = True
    elif data['Category'] == 'Bread':
        cat_Bread = True
    elif data['Category'] == 'Cheese':
        cat_Cheese = True
    elif data['Category'] == 'Extras':
        cat_Extras = True
    elif data['Category'] == 'Sauces':
        cat_Sauces = True
    elif data['Category'] == 'Veggies':
        cat_Veggies = True
    elif data['Category'] == 'Protein':
        cat_Protein = True
    elif data['Category'] == 'Seasonings':
        cat_Seasonings = True

    return pd.Series([cat_Sandwich, cat_Salad, cat_Breakfast, cat_Extra, cat_Wrap, cat_Bread
                    , cat_Cheese, cat_Extras, cat_Sauces, cat_Veggies, cat_Protein, cat_Seasonings])

In [ ]:
subway_cat = subway.copy()
subway_cat[['cat_Sandwich'
          , 'cat_Salad'
          , 'cat_Breakfast'
          , 'cat_Extra'
          , 'cat_Wrap'
          , 'cat_Bread'
          , 'cat_Cheese'
          , 'cat_Extras'
          , 'cat_Sauces'
          , 'cat_Veggies'
          , 'cat_Protein'
          , 'cat_Seasonings']] = subway_cat.apply(get_categories, axis=1)
subway_cat[subway_cat['Category'] == 'Breakfast'].head()

In [ ]:
subway_cat['Cal_per_g'] = subway_cat['Calories'] / subway_cat['Serving Size (g)']

mat = subway_cat[['Cal_per_g'
          , 'cat_Sandwich'
          , 'cat_Salad'
          , 'cat_Breakfast'
          , 'cat_Extra'
          , 'cat_Wrap'
          , 'cat_Bread'
          , 'cat_Cheese'
          , 'cat_Extras'
          , 'cat_Sauces'
          , 'cat_Veggies'
          , 'cat_Protein'
          , 'cat_Seasonings']].corr().abs()

mask = np.triu(np.ones_like(mat, dtype=bool))
mat_masked = mat.mask(mask)  # Pone a NaN todo lo que aparezca como True en la máscara

fig, ax = plt.subplots(figsize=(10,7)) 
sns.heatmap(mat_masked, annot=True, ax=ax)
plt.show()

Podemos destacar que las categorías 'Salad' y 'Veggies' son las que mayor relación tienen con las calorías, lo que es bastante sorprendente.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

# Clustering

En este apartado aplicaremos una estrategia de clustering al conjunto de datos con el objetivo de descubrir nuevas relaciones entre las columnas.

Para ello emplearemos el algoritmo Kmeans el cual trabaja iterativamente para asignar a cada muestra uno de los “K” grupos basado en sus características. Son agrupados en base a la similitud de sus columnas.

Los grupos se van definiendo de manera “orgánica”, es decir que se va ajustando su posición en cada iteración del proceso, hasta que converge el algoritmo. Una vez hallados los centroides debemos analizarlos para ver cuales son sus características únicas, frente a la de los otros grupos. Estos grupos son las etiquetas que genera el algoritmo.

Para reducir la complejidad del problema reduciremos el número de columnas del dataset.

In [ ]:
# Crear instancia del encoder
labelencoder = LabelEncoder()

clustering_dataset = subway.copy()

sns.pairplot(clustering_dataset, hue='Category',size=4,vars=['Calories', 'Cholesterol (mg)', 'Carbohydrates (g)', 'Sugars (g)'],kind='scatter')


Para seleccionar el número de clusters empleamos la regla del codo, la cual determina que el número óptimo de clusters se encuentra en el punto donde la curva deja de mejorar notablemente:

In [ ]:
clustering_dataset['Category'] = labelencoder.fit_transform(subway['Category'])

X = np.array(clustering_dataset[['Calories', 'Cholesterol (mg)', 'Carbohydrates (g)', 'Sugars (g)']])
y = np.array(clustering_dataset['Category'])

N_clusters = range(1, 10)
kmeans = [KMeans(n_clusters=i) for i in N_clusters]
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]

plt.plot(N_clusters,score)
plt.xlabel('Número del clusters')
plt.ylabel('Score')
plt.title('Curva del codo')
plt.show()

Elegimos K = 5, por lo que dividiremos el conjunto de datos en cinco clústers.

Vemos algunas de sus características en 3D:

In [ ]:
kmeans = KMeans(n_clusters=5).fit(X)
labels = kmeans.predict(X)

C = kmeans.cluster_centers_
colors=["orange","green","blue", "red", "brown"]
values=[]
for row in labels:
    values.append(colors[row])

fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=values,s=60)
ax.scatter(C[:, 0], C[:, 1], C[:, 2], marker='.', c=colors, s=1000)

Si agregamos el clúster al que pertenece cada clase al dataset, quedaría de la siguiente manera:

In [ ]:
clustering_dataset['Class']  = labels
clustering_dataset

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

# Regresión

Un modelo de regresión es un modelo matemático que busca determinar la relación entre una variable dependiente (Y), con respecto a otras variables, llamadas explicativas o independientes (X).

Existen pocos datos en nuestro dataset, por lo que vamos a utilizar Cross Validation con el fin de dividir los datos disponibles en subconjuntos, de modo que el ajuste y la predicción del modelo se puedan realizar en subconjuntos independientes. Debido a la baja cantidad de preparados, las pruebas con KFold() finalizaban a menudo sin llegar a converger a una solución adecuada, por lo que se ha optado en realizar K-Fold varias veces mediante RepeatedKFold() para conseguir dicha convergencia.

Aplicamos la regresión sobre la característica "Calories". Intentamos predecir qué cantidad de calorías tendrá un producto a partir del resto de valores nutricionales.

Para evaluar los modelos utilizamos la métrica de "Varianza capturada" ($R^2$), que nos proporciona la cantidad de variabilidad que tiene el modelo. Un $R^2 = 1$ se considera una predicción perfecta.

Los algoritmos de regresión aplicados son los siguientes:

- `KNeighborsRegressor`: Regresión basada en los K vecinos más cercanos.
- `LinearRegression`: Consiste en la obtención de la función de distribución lineal.
- `SVR`: Implementación basada en máquinas de vector soporte.
- `RandomForestRegressor`: Basado en hacer varios DecisionTrees(es un Ensemble). Se consigue mediante bootstrappingy selección aleatoria de dimensiones para cada uno de los árboles.


In [ ]:
#Hacemos copia auxiliar para no tener que leer todo el tiempo el mismo conjunto de datos
aux = subway.drop(columns=["Product", "Category"])
aux.dropna(axis=0, inplace=True)
aux = aux.reset_index(drop=True)

#Preparamos las etiquetas y las eliminamos 
labels = subway["Calories"]
aux.drop(["Calories"], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(aux, labels, test_size=0.2, random_state=33)

In [ ]:
N_SPLITS = 5
N_REPEATS = 10

#Declaramos cross validation
cv = RepeatedKFold(n_splits=N_SPLITS, n_repeats = N_REPEATS) 
print(cv)

# Variable acumuladora (cuatro posiciones, una por modelo)
mean_score = [0.0, 0.0, 0.0, 0.0]
for train_index, test_index in cv.split(X_train):

    # Seleccionamos los datos
    x_train_cv, y_train_cv = X_train.iloc[train_index], y_train.iloc[train_index]
    x_test_cv, y_test_cv = X_train.iloc[test_index], y_train.iloc[test_index]

    # Declaramos los modelos
    model_KNN = KNeighborsRegressor(n_neighbors=2)
    model_LiR = LinearRegression()
    model_SVR = SVR(max_iter=1000, C=50, kernel="poly", coef0=2)
    model_RF = RandomForestRegressor(criterion="mse", bootstrap=True)

    # Entrenamos los modelos
    model_KNN.fit(x_train_cv, y_train_cv)
    model_LiR.fit(x_train_cv, y_train_cv)
    model_SVR.fit(x_train_cv, y_train_cv)
    model_RF.fit(x_train_cv, y_train_cv)

    # Sumamos el resultado de la iteración
    mean_score[0] = mean_score[0] + model_KNN.score(x_test_cv,y_test_cv)
    mean_score[1] = mean_score[1] + model_LiR.score(x_test_cv,y_test_cv)
    mean_score[2] = mean_score[2] + model_SVR.score(x_test_cv,y_test_cv)
    mean_score[3] = mean_score[3] + model_RF.score(x_test_cv,y_test_cv)

    #print("Valor R2 (KNeighborsRegressor): ", model_KNN.score(x_test,y_test))
    #print("Valor R2 (LinearRegression): ", model_LiR.score(x_test,y_test))
    #print("Valor R2 (SVR): ", model_SVR.score(x_test,y_test))
    #print("Valor R2 (RandomForestRegressor): ", model_RF.score(x_test,y_test))


Vemos los resultados obtenidos para cada uno de los algoritmos de regresión:

In [ ]:
#Obtenemos la media de los resultados
print("RESULTADOS TRAIN:")
print("Resultado R2 (promedio) con Cross validation (KNeighborsRegressor): ", np.round(mean_score[0]/(N_SPLITS * N_REPEATS),2))
print("Resultado R2 (promedio) con Cross validation (LinearRegression): ", np.round(mean_score[1]/(N_SPLITS * N_REPEATS),2))
print("Resultado R2 (promedio) con Cross validation (SVR): ", np.round(mean_score[2]/(N_SPLITS * N_REPEATS),2))
print("Resultado R2 (promedio) con Cross validation (RandomForestRegressor): ", np.round(mean_score[3]/(N_SPLITS * N_REPEATS),2))

El modelo que mejor consigue capturar la varianza para el conjunto de datos de entrenamiento es `LinearRegression`, con un 92% de explicabilidad de las calorías respecto al resto de valores nutricionales.

In [ ]:
#Obtenemos las puntuaciones (R2) de los resultados de las predicciones
print("RESULTADOS TEST (datos no vistos previamente):")
print("Resultado R2 (promedio) con Cross validation (KNeighborsRegressor): ", np.round(model_KNN.score(X_test,y_test),2))
print("Resultado R2 (promedio) con Cross validation (LinearRegression): ", np.round(model_LiR.score(X_test,y_test),2))
print("Resultado R2 (promedio) con Cross validation (SVR): ", np.round(model_SVR.score(X_test,y_test),2))
print("Resultado R2 (promedio) con Cross validation (RandomForestRegressor): ", np.round(model_RF.score(X_test,y_test),2))

El mejor modelo se mantiene si los aplicamos sobre el conjunto de datos de test (datos no vistos previamente por el modelo). El `LinearRegression`, con un 94% de explicabilidad. Sin embargo, en este caso `SVR` adelanta a `RandomForestRegressor`, que realiza mejores predicciones sobre estos datos. En ambos casos, `KNeighborsRegressor` es el que más errores comete.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

# Clasificación



Cuando lo que pretendemos es identificar una clase o etiqueta asociada a una instancia, utilzamos modelos de clasificación.

En este caso utilizamos las métricas de accuracy y precision:
- `accuracy`: Todas las predicciones correctas respecto al total: (TP+TN)/(TP+FP+FN+TN)
- `precision`: Cuántos positivos son reales: TP/(TP+FP)

Los algoritmos de regresión aplicados son los siguientes:

- `LogisticRegression`: Regresión logística. Consiste en convertir una regresión lineal en una curva logarítmica. Utiliza un algoritmo OvR (One-vs-Rest) para el entrenamiento.
- `SVC`: Implementación basada en máquinas de vector soporte para clasificación. Se apoyan en las muestras para crear un margen que maximice la separación entre clases.


In [ ]:
#Hacemos copia auxiliar para no tener que leer todo el tiempo el mismo conjunto de datos
aux = subway.drop(columns=["Product"])

# Separamos etiquetas de dimensiones
labels = aux["Category"]
aux = aux.drop(columns=["Category"])

#Escalamos los datos
scaler = MinMaxScaler()
aux = pd.DataFrame(scaler.fit_transform(aux), columns=aux.columns)

X_train, X_test, y_train, y_test = train_test_split(aux, labels, test_size=0.2, random_state=33)

In [ ]:
N_SPLITS = 5
N_REPEATS = 10

#Declaramos cross validation
cv = RepeatedKFold(n_splits=N_SPLITS, n_repeats = N_REPEATS) 

#Para cada fold
#Variable acumuladora
mean_score = [0.0, 0.0]
iteration = 0
precision = [0, 0]
for train_index, test_index in cv.split(X_train):
     
     iteration = iteration + 1

     #Seleccionamos los datos
     x_train_cv, y_train_cv = X_train.iloc[train_index], y_train.iloc[train_index]
     x_test_cv, y_test_cv = X_train.iloc[test_index], y_train.iloc[test_index]

     #Declaramos modelo
     model_LR = LogisticRegression(C=100, max_iter=10000)
     model_SVC = SVC(gamma="auto", C=10000)

     model_LR.fit(x_train_cv, y_train_cv)
     model_SVC.fit(x_train_cv, y_train_cv)

     mean_score[0] = mean_score[0] + model_LR.score(x_test_cv, y_test_cv)
     mean_score[1] = mean_score[1] + model_SVC.score(x_test_cv, y_test_cv)

     # print("Iteration ", iteration)
     # print("Accuracy (LogisticRegression): ", np.round(model_LR.score(x_test_cv,y_test_cv),2))
     # print("Accuracy (SVC): ", np.round(model_SVC.score(x_test_cv,y_test_cv),2))

     # Calculamos precision en cada fold
     y_predicted_LR = model_LR.predict(x_test_cv)
     y_predicted_SVC = model_SVC.predict(x_test_cv)

     # Con precision_score podemos saber la precisión para cada una de las clases
     precision[0] = precision[0] + precision_score(y_test_cv, y_predicted_LR, average='weighted', labels=list(subway['Category'].unique()))
     precision[1] = precision[1] + precision_score(y_test_cv, y_predicted_SVC, average='weighted', labels=list(subway['Category'].unique()))

     # print("Precission (LogisticRegression): ", precision[0])
     # print("Precission (SVC): ", precision[1])
     # print("-----------------")


# Obtenemos la media de los resultados
# Accuracy:
resultado_LR_acc = np.round(mean_score[0]/(N_SPLITS * N_REPEATS),2)
resultado_SVC_acc = np.round(mean_score[1]/(N_SPLITS * N_REPEATS),2)
# Precision:
resultado_LR_pre = np.round(precision[0]/(N_SPLITS * N_REPEATS),2)
resultado_SVC_pre = np.round(precision[1]/(N_SPLITS * N_REPEATS),2)

# E imprimimos
print("RESULTADOS TRAIN:")
# Accuracy:
print("Accuracy con Cross validation (LogisticRegression): ", resultado_LR_acc) 
print("Precision con Cross validation (LogisticRegression): ", resultado_LR_pre)  
# Precision:
print("Accuracy con Cross validation (SVC): ", resultado_SVC_acc)    
print("Precision con Cross validation (SVC): ", resultado_SVC_pre)   
 

Obtenemos métricas similares para ambos modelos, manteniendo un accuracy y una precisión en torno al 80%.

In [ ]:
#Obtenemos las puntuaciones (R2) de los resultados de las predicciones
print("RESULTADOS TEST (datos no vistos previamente):")
print("Accuracy con Cross validation (LogisticRegression): ", np.round(model_LR.score(X_test,y_test),2))
print("Accuracy con Cross validation (SVC): ", np.round(model_SVC.score(X_test,y_test),2))

Para datos no vistos en el entrenamiento, el modelo `SVC` mantiene un accuracy similar al obtenido respecto a los datos de entrenamiento, por lo que el modelo ha aprendido a generalizar correctamente. Sin embargo, el accuracy del modelo de regresión logística ha descendido en aproximadamente un 10%, por lo que es superado por `SVC`. 

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

# Ensembles

Son combinaciones de modelos de aprendizaje simples que mejoran el rendimiento. Debemos tener en cuenta cuánto penaliza la combinación de estos modelos al tiempo de predicción (en paralelo si son completamente independientes unos de otros) respecto a los modelos simples. El esfuerzo adicional está justificado solo si el rendimiento es considerablemente mejor que el de los modelos simples.

Preparamos el conjunto de datos:


In [ ]:
#Hacemos copia auxiliar para no tener que leer todo el tiempo el mismo conjunto de datos
aux = subway.drop(columns=["Product"])

# Separamos etiquetas de dimensiones
labels = aux["Category"]
aux = aux.drop(columns=["Category"])

#Escalamos los datos
scaler = MinMaxScaler()
aux = pd.DataFrame(scaler.fit_transform(aux), columns=aux.columns)

X_train, X_test, y_train, y_test = train_test_split(aux, labels, test_size=0.2, random_state=33)

In [ ]:
X=X_train[['Calories', 'Carbohydrates (g)']]
y=y_train.to_numpy()

sc = StandardScaler()
sc.fit(X)
X_scaled = pd.DataFrame(sc.transform(X),columns = X.columns)

Y probamos con bagging y boosting:

## Bagging

Cuando usamos bagging, también combinamos varios modelos de machine learning. La forma de conseguir que los errores se compensen entre sí, es que cada modelo se entrena con subconjuntos del conjunto de entrenamiento. Estos subconjuntos se forman eligiendo muestras aleatoriamente (con repetición) del conjunto de entrenamiento.

Los modelos que vamos a combinar son de tipo `DecisionTreeClassifier`: crean una estructura de árbol basada en reglas lógicas. Parten de un nodo raíz (el que mejor separación aporte), hasta que llegan a las hojas, donde se tiene la etiqueta. Podemos especificar diferentes parámetros, donde dos de los más críticos son la profundidad máxima de los árboles, y el número máximo de nodos por hoja.

In [ ]:
from sklearn.ensemble import BaggingClassifier

num_models=10
bagging = BaggingClassifier(DecisionTreeClassifier(max_depth=10, max_leaf_nodes=10),n_estimators=num_models,max_samples=0.5, max_features=0.5)

In [ ]:
clf = bagging.fit(X_scaled.to_numpy(), y)

train_err = (clf.predict(X_scaled.to_numpy()) != y).mean()
print(f'Train error: {train_err:.1%}')

En este caso, cambiar la profundidad no nos ayuda demasiado a reducir el error. Lo que sí se ha notado es que aumentar el número de hojas hasta seis nos ha permitido reducir el error, aunque seguir aumentando el número máximo una vez alcanzado este punto ya no proporciona una mejora sustancial.

Visualizamos los árboles generados y las distintas lógicas aplicadas en cada uno de ellos:

In [ ]:
nrows=int(num_models/2)
ncols = 2

fig, axes = plt.subplots(figsize=(8, num_models*3),
                             nrows=nrows,
                             ncols=ncols,
                             sharex=True,
                             dpi=100)
for r in range(nrows):
  for c in range(ncols):
    i=r*ncols+c
    plot_tree(bagging.estimators_[i], class_names=list(subway['Category'].unique()), filled=True,ax=axes[r][c])
    axes[r][c].title.set_text("Model " + str (i) + " with feature " + str(bagging.estimators_features_[i][0]))

Para finalizar el análisis del ensemble, visualilzamos la matriz de confusión y observamos las métricas obtenidas.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = bagging.predict(X_test[['Calories', 'Carbohydrates (g)']])

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

print("Classification Report")
print(classification_report(y_test, predictions))

No hemos conseguido mejorar las predicciones que mediante la utilización de modelos simples. El ensemble ha aprendido a predecir sandwiches sobre todo. Sabemos que la categoría Sandwich es la que más se repite en el conjunto de datos, por lo que claramente se ha producido overfitting y las métricas obtenidas son bajas. 

Para solucionarlo, debemos aumentar las muestras o balancear el conjunto de datos. Sin embargo, tenemos categorías para las que solamente existe un producto, y el balanceo aquí se hace complicado.

## Boosting

En el boosting, cada modelo intenta arreglar los errores de los modelos anteriores. Por ejemplo, en el caso de clasificación, el primer modelo tratará de aprender la relación entre los atributos de entrada y el resultado. Seguramente cometerá algunos errores. Así que el segundo modelo intentará reducir estos errores. Esto se consigue dándole más peso a las muestras mal clasificadas y menos peso a las muestras bien clasificadas. 

In [ ]:
X=X_train[['Calories', 'Carbohydrates (g)']]
y=y_train.to_numpy()

sc = StandardScaler()
sc.fit(X)
X_scaled = pd.DataFrame(sc.transform(X),columns = X.columns)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradientboost = GradientBoostingClassifier(n_estimators=100,learning_rate=1, max_depth=1, random_state=0).fit(X_scaled, y)

train_err = (gradientboost.predict(X_scaled.to_numpy()) != y).mean()
print(f'Train error: {train_err:.1%}')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = gradientboost.predict(X_test[['Calories', 'Carbohydrates (g)']])

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

print("Classification Report")
print(classification_report(y_test, predictions))

Al igual que ocurría con bagging, los resultados no son todo lo buenos que esperábamos. El ensemble ha acabado con overfitting y necesitamos balancear o aumentar el conjunto de datos de entrenamiento para realizar mejores predicciones.

# Conclusiones

Hemos analizado el conjunto de datos y las relaciones entre sus características, y hemos logrado obtener cinco clústers o segmentos que nos van a permitir realizar análisis más específicos (líneas futuras).

Hemos logrado predecir las calorías de los preparados en función del resto de propiedades nutricionales de los preparados con resultados satisfactorios. Sin embargo, el conjunto de datos es bastante limitado y se hace difícil clasificar las categorías a las que pertenecen dichos preparados. Hemos visto que la relación de los valores nutricionales con las categorías es mucho menor que la obtenida con las calorías.

Por último, hemos realizado el ejercicio de combinar modelos simples de clasificación (ensembles) para intentar mejorar los resultados pobres en la predicción de la categoría, con resultados de nuevo poco convincentes. Como se ha comentado, necesitaríamos aumentar el número de muestras o añadir características adicionales para conseguir un mejor resultado.
